In [ ]:
from utils_package import *
from utils_package.loaders import build_loaders
from utils_package.net_UNET import Unet
from utils_package.constants import IN_CHANNELS, OUT_CHANNELS, LEARNING_RATE, MIN_EPOCHS, MAX_EPOCHS, DEVICE, EARLY_STOP_COUNT
from utils_package.train_function import train_loop, test_loop
from utils_package.plot import plot_batch, plot_loss, plot_avg_metric
from utils_package.utils import early_stopping, save_model, load_model, dc_loss

In [ ]:
model = Unet(IN_CHANNELS, OUT_CHANNELS).to(device = DEVICE)
loss_fn = dc_loss
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)
train_loader, test_loader = build_loaders()
checkpoint = "models/dice_loss.pth"

# ----- LOAD THE MODEL -----
if os.path.exists(checkpoint):
    model, optimizer, loss_fn = load_model(model, optimizer, checkpoint)

In [ ]:
def main():
    # ----- PLOT SAMPLE OF IMAGES AND TARGETS -----
    images, targets, classes = next(iter(train_loader))
    plot_batch(images[10::], targets[10::], classes[10::], 10, (30, 6))

    train_losses = []
    test_losses = []
    iou_scores = []
    dice_scores = []
    pixel_errors = []
    stop_count = 0

    for epoch in range(MAX_EPOCHS):
        avg_train_loss = train_loop(train_loader, model, optimizer, loss_fn, epoch, hpc=False)
        avg_test_loss, avg_iou_scores, avg_dice_scores, avg_pixel_error = test_loop(test_loader, model, loss_fn, hpc=False)

        train_losses.append(avg_train_loss)
        test_losses.append(avg_test_loss)
        iou_scores.append(avg_iou_scores)
        dice_scores.append(avg_dice_scores)
        pixel_errors.append(avg_pixel_error)

        # ----- PLOT EVALUATION METRICS -----
        plot_loss(train_losses, "Train Losses", "losses", path="plots/train_loss")
        plot_loss(test_losses, "Validation Losses", "losses", path="plots/test_loss")
        plot_avg_metric(iou_scores, "IoU Score Per Class", path = "plots/iou")
        plot_avg_metric(dice_scores, "Dice Score Per Class", path = "plots/dice")
        plot_avg_metric(pixel_errors, "Pixel Error Per Class", path = "plots/pixel_error")

        # ----- SAVE THE CHECKPOINT OF THE MODEL -----
        save_model(model.state_dict(), optimizer.state_dict(), loss_fn, checkpoint)

        # ----- CHECK EARLY STOPPING -----
        if early_stopping(test_losses): stop_count += 1
        else: stop_count = 0

        if stop_count >= EARLY_STOP_COUNT and epoch >= MIN_EPOCHS:
            break

In [ ]:
if __name__ == "__main__":
    main()